In [ ]:
# google-auth library provides tools for authenticating with Google APIs using various methods (like OAuth2 or service accounts)
# requests library is for making HTTP requests.
!pip install google-auth requests

In [ ]:
import requests
from google.oauth2 import service_account
from google.auth.transport.requests import Request  # Use this for token refresh

# Load the Service Account credentials (Ensure this file is uploaded to Fabric)
SERVICE_ACCOUNT_FILE = "./builtin/ardent-bridge-453606-i0-2006c1e37bf0.json"
SCOPES = ["https://www.googleapis.com/auth/drive"]

# Authenticate using Service Account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

# refresh token
if not credentials.valid:
    credentials.refresh(Request())

# Get the access token
access_token = credentials.token

# Define headers for API request
headers = {
    "Authorization": f"Bearer {access_token}"
}

print("Authentication successful!")

In [ ]:
import shutil
import os

url = "https://www.googleapis.com/drive/v3/files"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    files = response.json().get("files", [])
    for file in files:
        print(f"File Name: {file['name']}, File ID: {file['id']}")
        file_id = file['id']
        file_name = file['name']
        download_url = f"https://www.googleapis.com/drive/v3/files/{file_id}?alt=media"

        response = requests.get(download_url, headers=headers, stream=True)

        if response.status_code == 200:
            with open(file_name, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
            print(f"File '{file_name}' downloaded successfully!")

            # Define correct Fabric Lakehouse path
            lakehouse_folder = "/lakehouse/default/Files/"

            # Ensure the directory exists
            os.makedirs(lakehouse_folder, exist_ok=True)

            # Move the file
            shutil.move(file_name, os.path.join(lakehouse_folder, file_name))

            print(f"File '{file_name}' moved to Fabric Lakehouse successfully!")
        else:
            print("Error downloading file:", response.text)

else:
    print("Error retrieving file list:", response.text)
